In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def getOlx():
    olx_list = []
    for i in tqdm(range(1,101), desc='OLX'):
        r = requests.get(f'https://www.olx.com.br/imoveis/estado-sp?o={i}', headers=headers)
        content = r.content
        soup = BeautifulSoup(r.content, 'html.parser') 
        li_tags = soup.find_all('li')
        # Iterando sobre as tags <li> encontradas
        for li_tag in li_tags:
            # Verificando se a tag <li> contém um link
            if li_tag.find('a'):
                # Obtendo o valor do atributo "href" do link
                link = li_tag.find('a')['href']        
                if link.startswith('https://sp.olx.com.br'):
                    olx_list.append(link)
    return olx_list

def getLoft():
    temp = []
    loft_list = []
    for i in tqdm(range(1, 133),desc='Loft'):
            response = requests.get(f'https://loft.com.br/venda/imoveis/sp/sao-paulo?utm_source=google&utm_medium=cpc&utm_campaign=ins_01_br_001_sp_0001_sao-paulo_all_aw_search_conversion_broad_&utm_content=all_loft-pura&utm_id=1756086660&utm_placement=&utm_ad_id=576438774862&gclid=Cj0KCQjwjryjBhD0ARIsAMLvnF9KvelK1iKF7xoTJ3HS0KSWMhoDO0UlANc1gTfQEMGg5Bud0HiPZg4aAscOEALw_wcB&pagina={i}')
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')

            divs = soup.find_all('div', class_='MuiGrid-root')

            for div in divs:
                tag_a = div.find('a')
                if tag_a is not None:
                    link = tag_a['href']
                    if link.startswith('/imovel/'):
                        temp.append(link)
    loft_list = ['http://loft.com.br' + valor for valor in temp]
    return loft_list

## MAIN CODE
olx_list = getOlx()
loft_list = getLoft()

## get olx data


dados_gerais = []  # Lista para armazenar os dados de todas as URLs

for url in tqdm(olx_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'lxml') 

    h2 = soup.find('h2', class_='ad__sc-1wimjbb-1 hoHpcC sc-cooIXK cXlgiS')
    if h2:
        h2_value = h2.text.strip()
    else:
        h2_value = ''

    divs = soup.find_all('div', class_='sc-hmzhuo ad__sc-1f2ug0x-3 sSzeX sc-jTzLTM iwtnNi')  # Substitua 'nome_da_classe' pela classe real das divs

    dados_url = {}  # Dicionário para armazenar os dados de uma URL específica

    # Adicione a URL ao dicionário da URL específica com a chave 'URL'
    dados_url['URL'] = url

    # Adicione o valor da tag h2 ao dicionário da URL específica com a chave 'Valor'
    dados_url['Valor'] = h2_value

    # Itere sobre as divs encontradas
    for div in divs:
        # Inicialize um dicionário vazio para armazenar os valores da div atual
        dados = {}

        # Encontre todas as tags dt dentro da div atual
        tags_dt = div.find_all('dt')

        # Itere sobre as tags dt e obtenha os valores
        for dt in tags_dt:
            key = dt.text.strip()  # Use strip() para remover espaços em branco extras

            # Encontre a próxima tag dd
            dd = dt.find_next_sibling('dd')

            # Se a tag dd existir, obtenha o valor
            if dd:
                value = dd.text.strip()
            else:
                # Se não houver uma tag dd, encontre a próxima div com a classe específica
                proxima_div = div.find_next('div', class_='ad__sc-1f2ug0x-2 eSYIff')

                # Encontre a tag a dentro da próxima div
                a = proxima_div.find('a') if proxima_div else None

                # Obtenha o valor da tag a, se existir
                value = a.text.strip() if a else ''

            dados[key] = value

        # Adicione o dicionário da div atual ao dicionário da URL específica
        dados_url.update(dados)

    # Adicione o dicionário da URL específica à lista geral
    dados_gerais.append(dados_url)

# Converter a lista de dicionários em um DataFrame do pandas
df = pd.DataFrame(dados_gerais)

# Salvar o DataFrame em um arquivo Excel
df.to_excel('dados.xlsx', index=False)



Loft: 100%|██████████| 132/132 [00:27<00:00,  4.78it/s]


In [2]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


def getLoft():
    temp = []
    loft_list = []
    for i in tqdm(range(1, 133),desc='Loft'):
            response = requests.get(f'https://loft.com.br/venda/imoveis/sp/sao-paulo?utm_source=google&utm_medium=cpc&utm_campaign=ins_01_br_001_sp_0001_sao-paulo_all_aw_search_conversion_broad_&utm_content=all_loft-pura&utm_id=1756086660&utm_placement=&utm_ad_id=576438774862&gclid=Cj0KCQjwjryjBhD0ARIsAMLvnF9KvelK1iKF7xoTJ3HS0KSWMhoDO0UlANc1gTfQEMGg5Bud0HiPZg4aAscOEALw_wcB&pagina={i}')
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')

            divs = soup.find_all('div', class_='MuiGrid-root')

            for div in divs:
                tag_a = div.find('a')
                if tag_a is not None:
                    link = tag_a['href']
                    if link.startswith('/imovel/'):
                        temp.append(link)
    loft_list = ['http://loft.com.br' + valor for valor in temp]
    return loft_list

## MAIN CODE

loft_list = getLoft()

Loft: 100%|██████████| 132/132 [00:29<00:00,  4.47it/s]


In [14]:
import re

for url in tqdm(loft_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser') 
    desc = soup.find('h1', class_='MuiTypography-root jss197 jss174 jss183 MuiTypography-body1')

    padrao_categoria = r"^(\w+)"
    padrao_tipo = r"à\s(\w+)"
    padrao_bairro = r"em\s(.+?)\scom"
    padrao_tamanho = r"(\d+)\s*m²"
    padrao_quartos = r"(\d+)\squartos"
    padrao_vaga = r"(\d+)\svaga"

    dados = {}

    dados['categoria'] = re.search(padrao_categoria, desc.text).group(1)
    dados['tipo'] = re.search(padrao_tipo, desc.text).group(1)
    dados['bairro'] = re.search(padrao_bairro, desc.text).group(1)
    dados['tamanho'] = re.search(padrao_tamanho, desc.text).group(1)
    try:
        dados['quartos'] = re.search(padrao_quartos, desc.text).group(1)
    except:
        dados['quartos'] = '0'
    try:
        dados['vaga'] = re.search(padrao_vaga, desc.text).group(1)
    except:
        dados['vaga'] = '0'

print(dados)

{'categoria': 'Casa', 'tipo': 'venda', 'bairro': 'Chácara Klabin', 'tamanho': '295', 'quartos': '4', 'vaga': '3'}


In [51]:
import re

for url in tqdm(loft_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser') 
    desc = soup.find('h1', class_='MuiTypography-root jss197 jss174 jss183 MuiTypography-body1')
    title = soup.find('title')

    padrao_categoria = r"^(\w+)"
    padrao_tipo = r"à\s(\w+)"
    padrao_bairro = r"em\s(.+?)\scom"
    padrao_tamanho = r"(\d+)\s*m²"
    padrao_quartos = r"(\d+)\squartos"
    padrao_vaga = r"(\d+)\svaga"
    padrao_valor = r"R\$\s([\d.,]+)"

    dados = {}
    try:
        dados['categoria'] = re.search(padrao_categoria, desc.text).group(1)
    except:
        dados['categoria'] = 'Unknow'
    try:
        dados['tipo'] = re.search(padrao_tipo, desc.text).group(1)
    except:
        dados['tipo'] = 'Unknow'
    try:
        dados['bairro'] = re.search(padrao_bairro, desc.text).group(1)
    except:
        dados['bairro'] = 'Unknow'
    try:        
        dados['tamanho'] = re.search(padrao_tamanho, desc.text).group(1)
    except:
        dados['tamanho'] = '0'
    try:
        dados['quartos'] = re.search(padrao_quartos, desc.text).group(1)
    except:
        dados['quartos'] = '0'
    try:
        dados['vaga'] = re.search(padrao_vaga, desc.text).group(1)
    except:
        dados['vaga'] = '0'
    try:
        dados['valor'] = re.search(padrao_valor, title.text).group(1)
    except:
        dados['valor'] = '0'
    
    dados['url'] = url

    print(dados)


  0%|          | 2/5000 [00:00<09:44,  8.56it/s]

{'categoria': 'Casa', 'tipo': 'venda', 'bairro': 'Aricanduva', 'tamanho': '382', 'quartos': '3', 'vaga': '8', 'valor': '1.900.000', 'url': 'http://loft.com.br/imovel/casa-rua-sao-felix-de-cantalicio-aricanduva-sao-paulo-3-quartos-382m2/kj3fsq'}
{'categoria': 'Casa', 'tipo': 'venda', 'bairro': 'Perdizes', 'tamanho': '90', 'quartos': '3', 'vaga': '2', 'valor': '650.000', 'url': 'http://loft.com.br/imovel/casa-rua-tavares-bastos-perdizes-sao-paulo-3-quartos-90m2/15y1fe8'}


  0%|          | 3/5000 [00:00<09:48,  8.49it/s]

{'categoria': 'Apartamento', 'tipo': 'venda', 'bairro': 'Pari', 'tamanho': '69', 'quartos': '3', 'vaga': '2', 'valor': '827.999', 'url': 'http://loft.com.br/imovel/apartamento-rua-silva-teles-pari-sao-paulo-3-quartos-69m2/1dasle7'}


  0%|          | 5/5000 [00:00<13:12,  6.30it/s]

...

{'categoria': 'Apartamento', 'tipo': 'venda', 'bairro': 'Cambuci', 'tamanho': '33', 'quartos': '2', 'vaga': '0', 'valor': '380.000', 'url': 'http://loft.com.br/imovel/apartamento-rua-alexandrino-da-silveira-bueno-cambuci-sao-paulo-2-quartos-33m2/ev57gs'}


  1%|          | 28/5000 [00:11<34:15,  2.42it/s]

{'categoria': 'Apartamento', 'tipo': 'venda', 'bairro': 'Moema Índios', 'tamanho': '116', 'quartos': '3', 'vaga': '2', 'valor': '1.880.000', 'url': 'http://loft.com.br/imovel/apartamento-alameda-dos-anapurus-moema-indios-sao-paulo-3-quartos-116m2/1feduwo'}


  1%|          | 28/5000 [00:11<34:43,  2.39it/s]


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
import re
import pandas as pd

dados_imoveis = []

for url in tqdm(loft_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser')
    desc = soup.find('h1', class_='MuiTypography-root jss197 jss174 jss183 MuiTypography-body1')
    title = soup.find('title')

    padrao_categoria = r"^(\w+)"
    padrao_tipo = r"à\s(\w+)"
    padrao_bairro = r"em\s(.+?)\scom"
    padrao_tamanho = r"(\d+)\s*m²"
    padrao_quartos = r"(\d+)\squartos"
    padrao_vaga = r"(\d+)\svaga"
    padrao_valor = r"R\$\s([\d.,]+)"

    dados = {}

    try:
        dados['categoria'] = re.search(padrao_categoria, desc.text).group(1)
    except:
        dados['categoria'] = 'Unknow'
    try:
        dados['tipo'] = re.search(padrao_tipo, desc.text).group(1)
    except:
        dados['tipo'] = 'Unknow'
    try:
        dados['bairro'] = re.search(padrao_bairro, desc.text).group(1)
    except:
        dados['bairro'] = 'Unknow'
    try:
        dados['tamanho'] = re.search(padrao_tamanho, desc.text).group(1)
    except:
        dados['tamanho'] = '0'
    try:
        dados['quartos'] = re.search(padrao_quartos, desc.text).group(1)
    except:
        dados['quartos'] = '0'
    try:
        dados['vaga'] = re.search(padrao_vaga, desc.text).group(1)
    except:
        dados['vaga'] = '0'
    try:
        dados['valor'] = re.search(padrao_valor, title.text).group(1)
    except:
        dados['valor'] = '0'

    dados['URL'] = url

    dados_imoveis.append(dados)

# Cria o DataFrame
df = pd.DataFrame(dados_imoveis)

# Define a ordem das colunas
colunas = ['URL', 'Valor', 'CEP', 'Município', 'Bairro', 'Logradouro', 'Categoria', 'Vagas na garagem',
           'Quartos', 'Acomoda', 'Características', 'Tipo', 'Área construída', 'Banheiros',
           'Detalhes do imóvel', 'Condomínio', 'IPTU', 'Área útil', 'Detalhes do condomínio', 'Tamanho']

# Reorganiza as colunas
df = df[colunas]

# Exporta o DataFrame para um arquivo Excel
df.to_excel('dados_imoveis.xlsx', index=False)


In [48]:
test = ['http://loft.com.br/imovel/casa-rua-sao-felix-de-cantalicio-aricanduva-sao-paulo-3-quartos-382m2/kj3fsq',
 'http://loft.com.br/imovel/casa-rua-tavares-bastos-perdizes-sao-paulo-3-quartos-90m2/15y1fe8',
 'http://loft.com.br/imovel/apartamento-rua-silva-teles-pari-sao-paulo-3-quartos-69m2/1dasle7',
 'http://loft.com.br/imovel/apartamento-rua-do-estilo-barroco-santo-amaro-sao-paulo-4-quartos-284m2/oofj5l',
 'http://loft.com.br/imovel/apartamento-rua-davi-banderali-artur-alvim-sao-paulo-2-quartos-48m2/1hktuao']

In [54]:
len(dados_imoveis)

185